In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam

# ---------------- DEVICE ----------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ---------------- DATASET ----------------
class EMGDataset(Dataset):
    def __init__(self, x_path, y_path):
        X = np.load(x_path)        # (N, T, C)
        y = np.load(y_path)        # (N, num_classes)

        # Per-sample, per-channel normalization
        for i in range(X.shape[0]):
            for c in range(X.shape[2]):
                mu = X[i, :, c].mean()
                std = X[i, :, c].std() + 1e-8
                X[i, :, c] = (X[i, :, c] - mu) / std

        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(np.argmax(y, axis=1), dtype=torch.long)

        # 🔥 infer number of classes automatically
        self.num_classes = y.shape[1]

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# ---------------- CNN ENCODER ----------------
class CNNEncoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(6, 64, kernel_size=10),
            nn.ReLU(),

            nn.Conv1d(64, 64, kernel_size=10),
            nn.ReLU(),
            nn.MaxPool1d(3),

            nn.Conv1d(64, 256, kernel_size=10),
            nn.ReLU(),

            nn.Conv1d(256, 256, kernel_size=10),
            nn.ReLU(),

            nn.AdaptiveAvgPool1d(1)   # (B, 256, 1)
        )

    def forward(self, x):
        x = x.permute(0, 2, 1)      # (B, C, T)
        x = self.net(x)
        return x.squeeze(-1)        # (B, 256)

# ---------------- FULL MODEL ----------------
class FullModel(nn.Module):
    def __init__(self, encoder, num_classes):
        super().__init__()
        self.encoder = encoder
        self.classifier = nn.Linear(256, num_classes)

    def forward(self, x):
        emb = self.encoder(x)
        logits = self.classifier(emb)
        return logits

# ---------------- EARLY STOPPING ----------------
class EarlyStopping:
    def __init__(self, patience=7):
        self.patience = patience
        self.best_acc = 0.0
        self.counter = 0
        self.best_state = None

    def step(self, acc, model):
        if acc > self.best_acc:
            self.best_acc = acc
            self.counter = 0
            self.best_state = {
                k: v.detach().cpu().clone()
                for k, v in model.state_dict().items()
            }
            return False
        else:
            self.counter += 1
            return self.counter >= self.patience

    def restore(self, model):
        model.load_state_dict(self.best_state)

# ---------------- TRAIN & EVAL ----------------
def train_and_eval(model, train_loader, test_loader,
                   epochs=500, patience=7):

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=5e-4)
    early_stopper = EarlyStopping(patience)

    for epoch in range(epochs):
        model.train()
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)

            logits = model(x)
            loss = criterion(logits, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # ----- validation -----
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for x, y in test_loader:
                x, y = x.to(device), y.to(device)
                preds = model(x).argmax(dim=1)
                correct += (preds == y).sum().item()
                total += y.size(0)

        val_acc = correct / total
        print(f"Epoch {epoch+1:02d} | Val Acc: {val_acc:.4f}")

        if early_stopper.step(val_acc, model):
            print("Early stopping")
            break

    early_stopper.restore(model)
    return early_stopper.best_acc

# ---------------- MAIN LOOP ----------------
NUM_CLASSES = 47    # 🔁 change to 62 if needed
BASE = f"models/Data/Data/{NUM_CLASSES}_classes/UserDependenet"

accs = []

for split in range(1, 11):
    print(f"\n===== Split {split} =====")

    train_ds = EMGDataset(
        f"{BASE}/Train/X_train_{split}.npy",
        f"{BASE}/Train/y_train_{split}.npy"
    )
    test_ds = EMGDataset(
        f"{BASE}/Test/X_test_{split}.npy",
        f"{BASE}/Test/y_test_{split}.npy"
    )

    train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
    test_loader = DataLoader(test_ds, batch_size=128)

    model = FullModel(
        CNNEncoder(),
        num_classes=train_ds.num_classes   # 🔥 auto
    ).to(device)

    acc = train_and_eval(model, train_loader, test_loader)
    accs.append(acc)

    print(f"Best accuracy (split {split}): {acc:.4f}")

print("\nFINAL AVERAGE ACCURACY (CNN + CCE):", np.mean(accs))

Using device: cuda

===== Split 1 =====
Epoch 01 | Val Acc: 0.1723
Epoch 02 | Val Acc: 0.3702
Epoch 03 | Val Acc: 0.5043
Epoch 04 | Val Acc: 0.5713
Epoch 05 | Val Acc: 0.6032
Epoch 06 | Val Acc: 0.6479
Epoch 07 | Val Acc: 0.6670
Epoch 08 | Val Acc: 0.7000
Epoch 09 | Val Acc: 0.7181
Epoch 10 | Val Acc: 0.7340
Epoch 11 | Val Acc: 0.7489
Epoch 12 | Val Acc: 0.7447
Epoch 13 | Val Acc: 0.7553
Epoch 14 | Val Acc: 0.7394
Epoch 15 | Val Acc: 0.7766
Epoch 16 | Val Acc: 0.7798
Epoch 17 | Val Acc: 0.7883
Epoch 18 | Val Acc: 0.7883
Epoch 19 | Val Acc: 0.7936
Epoch 20 | Val Acc: 0.7766
Epoch 21 | Val Acc: 0.8000
Epoch 22 | Val Acc: 0.7936
Epoch 23 | Val Acc: 0.8011
Epoch 24 | Val Acc: 0.8043
Epoch 25 | Val Acc: 0.7872
Epoch 26 | Val Acc: 0.8011
Epoch 27 | Val Acc: 0.8106
Epoch 28 | Val Acc: 0.7904
Epoch 29 | Val Acc: 0.7926
Epoch 30 | Val Acc: 0.8074
Epoch 31 | Val Acc: 0.7926
Epoch 32 | Val Acc: 0.8085
Epoch 33 | Val Acc: 0.8021
Epoch 34 | Val Acc: 0.8191
Epoch 35 | Val Acc: 0.8053
Epoch 36 | Val 